In [2]:
import ml_collections
from ml_collections import config_flags
from absl import app

import sys
sys.path.append('/home/jwiers/deeprisk/new_codebase/enf-biobank')


import jax
import jax.numpy as jnp
import optax
import logging

import matplotlib.pyplot as plt

import wandb

# Custom imports
from experiments.datasets import get_dataloaders
from enf.model import EquivariantNeuralField
from enf.bi_invariants import TranslationBI
from enf.utils import create_coordinate_grid, initialize_latents

from experiments.downstream_models.transformer_enf import TransformerClassifier

In [10]:
transformer = TransformerClassifier(
        hidden_size=768,
        depth=12,
        num_heads=12,
        mlp_ratio=4,
        num_classes=10,
    )

key = jax.random.PRNGKey(0)

key, subkey = jax.random.split(key)

batch_size = 1
n_latents_slice = 128
h_dim = 32
t=6
z=5
dims = 4


n_latents = n_latents_slice * t * z 



pose_vector = jax.random.normal(subkey, (batch_size, n_latents, dims))
context_vector = jax.random.normal(subkey, (batch_size, n_latents, h_dim))
gaussian_vector = jax.random.normal(subkey, (batch_size, n_latents, 1))

print("pose_vector shape: ", pose_vector.shape)
print("context_vector shape: ", context_vector.shape)
print("gaussian_vector shape: ", gaussian_vector.shape)

print(f"sample dimensionality = {n_latents * h_dim}")

z = (pose_vector, context_vector, gaussian_vector)

params = transformer.init(key, *z)



pose_vector shape:  (1, 3840, 4)
context_vector shape:  (1, 3840, 32)
gaussian_vector shape:  (1, 3840, 1)
sample dimensionality = 122880


In [11]:
# Test forward and backward

logits = transformer.apply(params, *z)

def test_loss(params):
    return 0.01

# Get gradients
loss, grads = jax.value_and_grad(test_loss)(transformer_params)